## MODULES

In [141]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import wrapcauchy 
from scipy.stats import cauchy 

## FUNCTION FOR TURNING ANGLES

In [142]:
def calculate_turning_angles(df):
    """Calcula los ángulos de giro entre cada tres puntos consecutivos de una trayectoria."""
    angles = []
    for i in range(2, len(df)):
       
        v1 = np.array([df.iloc[i-1]['x_pos'] - df.iloc[i-2]['x_pos'], df.iloc[i-1]['y_pos'] - df.iloc[i-2]['y_pos']])
        v2 = np.array([df.iloc[i]['x_pos'] - df.iloc[i-1]['x_pos'], df.iloc[i]['y_pos'] - df.iloc[i-1]['y_pos']])
        
        # Ángulo de giro
        angle = np.arctan2(v2[1], v2[0]) - np.arctan2(v1[1], v1[0])
        angles.append(angle) 
    
    return np.array(angles)

## READ CSV

In [143]:

CRW_trajectory_1 = pd.read_csv('crw_6_6.csv')
CRW_trajectory_2 = pd.read_csv('crw_6_9.csv') 

## CALCULATE TURNING ANGLES

In [144]:
turning_angles_1 = calculate_turning_angles(CRW_trajectory_1)
turning_angles_2 = calculate_turning_angles(CRW_trajectory_2)

## CAUCHY

In [145]:
x_vals = np.linspace(-np.pi, np.pi, 1000)

cauchy_dist_1 = cauchy.pdf(x_vals,scale=0.2)  
cauchy_dist_2 = cauchy.pdf(x_vals,scale=0.1)  

## PLOT

In [146]:
fig = go.Figure()


fig.add_trace(go.Histogram(
    x=turning_angles_1,
    nbinsx=1000,
    histnorm='probability density',
    name='observed_0.6',
    marker=dict(color='blue', opacity=0.5)
))

fig.add_trace(go.Histogram(
    x=turning_angles_2,
    nbinsx=1000,
    histnorm='probability density',
    name='observed_0.9',
    marker=dict(color='red', opacity=0.5)
))

fig.add_trace(go.Scatter(
    x=x_vals,
    y=cauchy_dist_1,
    mode='lines',
    name='cauchy_0.6',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=x_vals,
    y=cauchy_dist_2,
    mode='lines',
    name='cauchy_0.9',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Turning-angle Distribution: Observed vs Cauchy',
    xaxis_title='Turning Angle',
    yaxis_title='Probability Density',
    barmode='overlay',
    showlegend=True,
    xaxis_range=[-np.pi, np.pi] 
)

fig.show()

